<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Example_WANDB_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!python -m pip install -q wandb

In [22]:
# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

In [23]:
# function to count sequence length
def count_seq_length(df: pd.DataFrame, seq_col: str) -> pd.Series:
    pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
    df[seq_col].replace(pattern, "", inplace= True)
    return df[seq_col].str.len()

In [24]:
# count sequence lengths
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
train_set = pd.read_csv(TRAIN_DATAPATH)
counts = count_seq_length(pd.read_csv(TRAIN_DATAPATH), "modified_sequence")

In [25]:
# Initialize WANDB
PROJECT = 'Example_WANDB_Support'
RUN = "run_4"
wandb.init(project = PROJECT, name = RUN)

In [26]:
# log table to wandb
c = counts.to_frame()
table = wandb.Table(dataframe = c)

In [27]:
# create custom plot and log it
field = list(train_set.columns.values)[0]

hist = wandb.plot_table(
  vega_spec_name="master_praktikum/hist_pep_len",
  data_table = table,
  fields = {"value" : field}
)
wandb.log({"Histogram": hist})

In [28]:
# finish run
wandb.finish()

In [33]:
# create report
report = wr.Report(
    project = PROJECT,
    title = "Test_Title",
    description = "Test_Description"
)
report.save()

Report(project='Example_WANDB_Support', entity='master_praktikum', title='Test_Title', description='Test_Description', width='readable')

In [34]:
# trying to add a custom plot to the report, using the data previously logged into the table
# this is where i can not find a solution to achieve what i want which is adding the same exact
# custom plot previously created to the report
# since the custom chart is already displayed i thin my issue is in the query, but
# i am not sure on how to access the logged table with the query
panels_data=[
  wr.MarkdownPanel("Hello *italic* **bold** $e=mc^2$ `something`"),
  wr.CustomChart(
    query={'summary':{"keys": [field]}},
    chart_name='master_praktikum/hist_pep_len',
    chart_fields={'value': field}
  ),
]

In [35]:
ENTITY = wandb.apis.PublicApi().default_entity

data_hists = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT),
    ],
    panels=
      panels_data

)

report.blocks = report.blocks[:1] +[wr.H1("Data metrics"), data_hists] + report.blocks[1:]

report.save()

Report(project='Example_WANDB_Support', entity='master_praktikum', title='Test_Title', description='Test_Description', width='readable', blocks=[H1(text=['Data metrics']), PanelGrid(runsets=[Runset(entity='master_praktikum', project='Example_WANDB_Support', name='Run set', query='', filters={'$or': [{'$and': []}]}, order=['-CreatedTimestamp'])], panels=[MarkdownPanel(markdown='Hello *italic* **bold** $e=mc^2$ `something`'), CustomChart(query={'runSets': {'runSets': '${runSets}', 'limit': 500}}, chart_name='master_praktikum/hist_pep_len', chart_fields={'value': 'modified_sequence'})])])